### Experiment: Pre-processing and frequency sampling

**Question**: Is it possible to train a model on unpreprocessed EEG data and still attain similar performance levels?

**Hypothesis**: The model will perform worse, but if still similar then the added value of not having to (manually) preprocess EEG data is very valuable and opens up a multitude of applications.

**Result**:

#### Part 1: Preparing data
To use hmp.utils.read_mne_data() and epoch the information, the files should be in .fif format, this replicates automated preprocessing as done in https://github.com/GWeindel/hsmm_mvpy/blob/main/tutorials/sample_data/eeg/0022.ipynb excepting resampling to 100Hz

In [1]:
import mne
from pathlib import Path
import hsmm_mvpy as hmp
import pandas as pd
import numpy as np
import xarray as xr
from shared.data import add_stage_dimension

2023-10-16 16:27:02.985116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 16:27:03.584778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set up paths and file locations
data_path = Path("/mnt/d/thesis/sat1/")
behavioral_data_path = data_path / "ExperimentData/ExperimentData"
output_path = Path("data/sat1/unpreprocessed")

subj_ids = [
    subj_id.name.split("-")[1][:4] for subj_id in (data_path / "eeg4").glob("*.vhdr")
]
subj_files = [
    str(output_path / f"unprocessed_{subj_id}_epo.fif") for subj_id in subj_ids
]
behavioral_files = [
    str(behavioral_data_path / f"{subj_id}-cnv-sat3_ET.csv") for subj_id in subj_ids
]

In [ ]:
# Replacing preprocessing done in https://github.com/GWeindel/hsmm_mvpy/blob/main/tutorials/sample_data/eeg/0022.ipynb
# with only the necessary (non-manual) parts, like adding metadata for processing in HMP package, more info in link above
for subject_id in subj_ids:
    print(f"Processing subject: {subject_id}")
    subject_id_short = subject_id.replace("0", "")
    raw = mne.io.read_raw_brainvision(
        data_path / "eeg4" / f"MD3-{subject_id}.vhdr", preload=False
    )
    raw.set_channel_types(
        {"EOGh": "eog", "EOGv": "eog", "A1": "misc", "A2": "misc"}
    )  # Declare type to avoid confusion with EEG channels
    raw.rename_channels({"FP1": "Fp1", "FP2": "Fp2"})  # Naming convention
    raw.set_montage("standard_1020")  # Standard 10-20 electrode montage
    raw.rename_channels({"Fp1": "FP1", "Fp2": "FP2"})

    behavioral_path = behavioral_data_path / f"{subject_id}-cnv-sat3_ET.csv"
    behavior = pd.read_csv(behavioral_path, sep=";")[
        [
            "stim",
            "resp",
            "RT",
            "cue",
            "movement",
        ]
    ]
    behavior["movement"] = behavior.apply(
        lambda row: "stim_left"
        if row["movement"] == -1
        else ("stim_right" if row["movement"] == 1 else np.nan),
        axis=1,
    )
    behavior["resp"] = behavior.apply(
        lambda row: "resp_left"
        if row["resp"] == 1
        else ("resp_right" if row["resp"] == 2 else np.nan),
        axis=1,
    )
    # Merging together the exeperimental conditions info to have the format condition/stimulus/response
    behavior["trigger"] = (
        behavior["cue"] + "/" + behavior["movement"] + "/" + behavior["resp"]
    )
    # Filtering out < 300 and > 3000 Reaction times
    behavior["RT"] = behavior.apply(
        lambda row: 0
        if row["RT"] < 300
        else (0 if row["RT"] > 3000 else float(row["RT"]) / 1000),
        axis=1,
    )
    epochs = mne.io.read_epochs_fieldtrip(
        data_path / "eeg1" / f"data{subject_id_short}.mat", info=raw.info
    )
    epochs.rename_channels({"FP1": "Fp1", "FP2": "Fp2"})  # Naming convention
    epochs.set_montage("easycap-M1")
    epochs.filter(1, 35)  # Bandwidth filter from van Maanen, Portoles & Borst (2021)
    epochs.crop(tmin=-0.250)
    epochs.set_eeg_reference("average")
    epochs.metadata = behavior
    epochs.save(
        output_path / f"unprocessed_{subject_id}_epo.fif", overwrite=True, verbose=False
    )  # Saving EEG mne format

In [4]:
output_path_data = Path("data/sat1/data_unprocessed_500hz.nc")
# Run if data_unprocessed.nc does not exist or should be rewritten
data = hmp.utils.read_mne_data(
    subj_files,
    epoched=True,
    lower_limit_RT=0.2,
    upper_limit_RT=2,
    verbose=False,
    subj_idx=subj_ids,
    rt_col="RT",
)
data.to_netcdf(output_path_data)

Processing participant data/sat1/unpreprocessed/unprocessed_0001_epo.fif's epoched eeg
198 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0001_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0002_epo.fif's epoched eeg
200 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0002_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0003_epo.fif's epoched eeg
191 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0003_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0004_epo.fif's epoched eeg
200 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0004_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0005_epo.fif's epoched eeg
190 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0005_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0006_epo.fif's epoched eeg
200 trials were retaine

In [5]:
output_path_data = Path("data/sat1/data_unprocessed_100hz.nc")
# Run if data_unprocessed.nc does not exist or should be rewritten
data = hmp.utils.read_mne_data(
    subj_files,
    epoched=True,
    lower_limit_RT=0.2,
    upper_limit_RT=2,
    sfreq=100,
    verbose=False,
    subj_idx=subj_ids,
    rt_col="RT",
)
data.to_netcdf(output_path_data)

Processing participant data/sat1/unpreprocessed/unprocessed_0001_epo.fif's epoched eeg
198 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0001_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0002_epo.fif's epoched eeg
200 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0002_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0003_epo.fif's epoched eeg
191 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0003_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0004_epo.fif's epoched eeg
200 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0004_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0005_epo.fif's epoched eeg
190 trials were retained for participant data/sat1/unpreprocessed/unprocessed_0005_epo.fif
Processing participant data/sat1/unpreprocessed/unprocessed_0006_epo.fif's epoched eeg
200 trials were retaine

#### Use information from stage_data to split unprocessed data

##### 500Hz

In [2]:
data_path = Path("data/sat1/stage_data.nc")
merge_dataset = xr.load_dataset(Path("data/sat1/data_unprocessed_500hz.nc"))
output_data = add_stage_dimension(data_path, merge_dataset)

Finding stage changes
17 0 response
<xarray.DataArray 'labels' (labels: 1)>
array(['response'], dtype='<U8')
Coordinates:
  * labels   (labels) <U8 'response'
23
8

21 28 response
<xarray.DataArray 'labels' (labels: 1)>
array(['response'], dtype='<U8')
Coordinates:
  * labels   (labels) <U8 'response'
53
13

Combining segments


KeyboardInterrupt: 

In [7]:
output_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")
output_data.to_netcdf(output_path)

##### 100Hz

In [8]:
data_path = Path("data/sat1/stage_data.nc")
merge_dataset = xr.load_dataset(Path("data/sat1/data_unprocessed_100hz.nc"))
output_data = add_stage_dimension(data_path, merge_dataset)

Finding stage changes
Combining segments


In [9]:
output_path = Path("data/sat1/split_stage_data_unprocessed_100hz.nc")
output_data.to_netcdf(output_path)

### Part 2: Experiment

In [1]:
import tensorflow as tf
import gc
from pathlib import Path
from shared.data import add_stage_dimension
from shared.training import split_data_on_participants, train_and_evaluate, k_fold_cross_validate, compile_kwargs
from shared.normalization import *
from shared.models import SAT1Base, SAT1Topological, SAT1Deep
from shared.utilities import print_results
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-10-16 13:35:11.961822: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 13:35:12.676477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-16 13:35:14.636565: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 13:35:14.661612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 13:35:14.661684: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_exec

env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


r/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 13:35:15.192557: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 13:35:15.192639: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-16 13:35:15.192647: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1726] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2023-10-16 13:35:15.192710: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to r

In [2]:
logs_path = Path("logs/exp_preprocessing/")

##### 2a: Processed 100Hz (control)

In [3]:
data_path = Path("data/sat1/split_stage_data.nc")
data = xr.load_dataset(data_path)

In [4]:
tf.keras.backend.clear_session()
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)
train_kwargs = {
    "logs_path": logs_path,
    "additional_info": {"preprocessing": "default_100hz"},
    "additional_name": f"preprocessing-default_100hz",
}
results = k_fold_cross_validate(
    data, model, 5, normalization_fn=norm_min1_to_1, train_kwargs=train_kwargs
)
print_results(results)
del model
gc.collect()

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/20


2023-10-16 13:35:20.327635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-16 13:35:20.958524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-16 13:35:20.960666: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9880014400 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-16 13:35:20.960690: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-16 13:35:20.964602: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-16 13:35:21.062440: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

KeyboardInterrupt: 

##### 2b: Unprocessed 100Hz

In [3]:
data_path = Path("data/sat1/split_stage_data_unprocessed_100hz.nc")
data = xr.load_dataset(data_path)

In [4]:
tf.keras.backend.clear_session()
model = SAT1Base(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)
train_kwargs = {
    "logs_path": logs_path,
    "additional_info": {"preprocessing": "unprocessed_100hz"},
    "additional_name": f"preprocessing-unprocessed_100hz",
}
results = k_fold_cross_validate(
    data, model, 5, normalization_fn=norm_min1_to_1, train_kwargs=train_kwargs
)
print_results(results)
del model
gc.collect()

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/10


2023-10-16 13:27:15.193660: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-16 13:27:16.107853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-16 13:27:16.110156: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5587931d77a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-16 13:27:16.110180: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-16 13:27:16.114030: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-16 13:27:16.210419: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

999/999 [==============================] - 17s 13ms/step - loss: 0.9601 - accuracy: 0.6227 - val_loss: 0.7204 - val_accuracy: 0.7413
Epoch 2/10
999/999 [==============================] - 13s 13ms/step - loss: 0.6628 - accuracy: 0.7553 - val_loss: 0.5544 - val_accuracy: 0.7966
Epoch 3/10
999/999 [==============================] - 13s 13ms/step - loss: 0.5639 - accuracy: 0.7889 - val_loss: 0.5042 - val_accuracy: 0.8150
Epoch 4/10
999/999 [==============================] - 13s 13ms/step - loss: 0.5248 - accuracy: 0.8052 - val_loss: 0.4940 - val_accuracy: 0.8147
Epoch 5/10
999/999 [==============================] - 13s 13ms/step - loss: 0.4889 - accuracy: 0.8203 - val_loss: 0.4835 - val_accuracy: 0.8190
Epoch 6/10
999/999 [==============================] - 13s 13ms/step - loss: 0.4620 - accuracy: 0.8303 - val_loss: 0.4493 - val_accuracy: 0.8347
Epoch 7/10
999/999 [==============================] - 13s 13ms/step - loss: 0.4315 - accuracy: 0.8400 - val_loss: 0.4536 - val_accuracy: 0.8349
Epo

KeyboardInterrupt: 

##### 2c: Unprocessed 500Hz

In [14]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")
data = xr.load_dataset(data_path)

In [15]:
tf.keras.backend.clear_session()
model = SAT1Deep(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)
train_kwargs = {
    "logs_path": logs_path,
    "additional_info": {"preprocessing": "unprocessed_500hz"},
    "additional_name": f"preprocessing-unprocessed_500hz",
}
results = k_fold_cross_validate(
    data, model, 5, normalization_fn=norm_min1_to_1, train_kwargs=train_kwargs
)
print_results(results)
del model
gc.collect()

Fold 1: test fold: ['0009' '0017' '0001' '0024' '0012']
Epoch 1/10


2023-10-13 14:58:39.461825: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


999/999 [==============================] - 49s 45ms/step - loss: 1.2078 - accuracy: 0.4944
Epoch 2/10
999/999 [==============================] - 45s 45ms/step - loss: 1.0380 - accuracy: 0.5850
Epoch 3/10
999/999 [==============================] - 44s 44ms/step - loss: 0.8440 - accuracy: 0.6839
Epoch 4/10
999/999 [==============================] - 44s 43ms/step - loss: 0.7151 - accuracy: 0.7388
Epoch 5/10
999/999 [==============================] - 44s 43ms/step - loss: 0.6575 - accuracy: 0.7577
Epoch 6/10
999/999 [==============================] - 44s 44ms/step - loss: 0.6195 - accuracy: 0.7760
Epoch 7/10
999/999 [==============================] - 43s 43ms/step - loss: 0.5738 - accuracy: 0.7956
Epoch 8/10
999/999 [==============================] - 44s 43ms/step - loss: 0.5430 - accuracy: 0.8089
Epoch 9/10
999/999 [==============================] - 44s 43ms/step - loss: 0.5072 - accuracy: 0.8171
Epoch 10/10
251/251 [==============================] - 3s 11ms/step
Fold 1: accuracy: 0.81225

KeyboardInterrupt: 

In [13]:
# View results in Tensorboard
! tensorboard --logdir logs/exp_preprocessing/

2023-09-25 13:59:04.226414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.13.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


: 